In [24]:
# imports
import pickle
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

In [25]:
# load filtered df
# ???
# load final dataset
df = pd.read_csv('../raw_data/full_dataset.csv')

In [99]:
df['genres'].describe

<bound method NDFrame.describe of 0                       ['classical', 'early romantic era']
1                                ['tango', 'vintage tango']
2         ['classical', 'early romantic era', 'polish cl...
3                       ['classical', 'early romantic era']
4                        ['classical', 'late romantic era']
5                                ['tango', 'vintage tango']
6         ['classical', 'post-romantic era', 'russian ro...
7         ['classical', 'early romantic era', 'polish cl...
8         ['american modern classical', 'classical', 'ea...
9                       ['classical', 'early romantic era']
10        ['classical', 'classical era', 'early romantic...
11        ['british modern classical', 'early modern cla...
12        ['early modern classical', 'impressionism', 'l...
13                               ['tango', 'vintage tango']
14                                          ['galante era']
15                               ['tango', 'vintage tango']
16    

In [26]:
type(df['duration_min'][0])

numpy.float64

In [3]:
# # explore data
# df.describe()

In [63]:
# functionize seeding & feature selection
def get_seed_features(df):
    '''Get random seed track & features'''
    seed = df.sample()
    #tempo = seed['scaled_tempo'].iat[0] 
    #loudness = seed['scaled_loudness'].iat[0]
    #da = seed['danceability'].iat[0]
    #energy = seed['energy'].iat[0]
    seed_df= seed[['scaled_tempo','scaled_loudness','danceability','energy']]
    return seed_df#, popularity

In [64]:
get_seed_features(df)

,scaled_tempo,scaled_loudness,danceability,energy
62313,0.372474,0.790569,0.76,0.99


In [65]:
# functionize trainig
def train_knn(df):
    '''Train the model'''
    features_names = ['scaled_tempo', 'scaled_loudness', 'danceability', 'energy'] # 'scaled_year', 'popularity_binned'
    X = df[features_names]
    y = df['track_id']
    model = KNeighborsRegressor(algorithm='kd_tree', n_jobs=-1).fit(X, y)
    return model

In [66]:
# train model
model = train_knn(df)
# pickle model
with open("../peak/knn_trained.pkl", "wb") as file:
    pickle.dump(model, file)   

In [69]:
# load trained model
knn_trained = pickle.load(open("../peak/knn_trained.pkl","rb")) #file is in peak/peak

In [68]:
# get seed features
seed_df = get_seed_features(df)

In [71]:
def get_k_dist_ind(knn_trained, seed_df):
    '''Get model output: distances & indices'''
    # set output & k
    knn_out, k = [], 100
    # get trained model output for k: distances & indices
    knn_out = knn_trained.kneighbors(seed_df, n_neighbors=k)
    return knn_out

In [74]:
# get output: distances & indices
knn_out = get_k_dist_ind(knn_trained,seed_df)
# get indices
ind = knn_out[1][0].tolist()

In [96]:
# k=100 recommendations df
recs = df.iloc[ind] 
total = recs[recs['duration_min'].cumsum() <= 30]
total['duration_min'].sum()

29.06